# 基于模式匹配的机器人对话匹配

rules:
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Saying:
Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)

In [23]:
def is_variable(pat):
    "return ?P 开头"
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:]) 

def pat_match(pattern,saying):
    "define a  simple procedure 模式匹配"
    #递归的出口条件
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    else:
        if pattern[0] != saying[0]:
            return []
        else:
            return pat_match(pattern[1:], saying[1:])

def  pat_to_dict(patterns):
    return {k:v for k,v in patterns}

def subsitite(pat_dic,saying):
    if not saying: return []
    return [pat_dic.get(saying[0],saying[0])] + subsitite(pat_dic,saying[1:])


    
        


In [79]:
got_patterns = pat_to_dict(pat_match('I want ?X and ?Y'.split(), 'I want iPhone and computer'.split()))

In [80]:
' '.join(subsitite(got_patterns,'why do you want ?X and ?Y'.split()))

'why do you want iPhone and computer'

In [26]:

# try another example
john_pat = pat_to_dict(pat_match('?P needs ?X'.split(), "John needs vacation".split()))

In [27]:
' '.join(subsitite(john_pat,'why do ?P want ?X'.split()))

'why do John want vacation'

In [90]:
# now we define reponse_dictionary
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}


## 问题1

#### 编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [29]:
import random

In [30]:
# 问题1

def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    for k,v in rules.items():
        pat_mat = pat_match(k.split(), saying.split())
        print(pat_mat)
        if len(pat_mat) != 0:
            parsed_rules = pat_to_dict(pat_mat)
            response = ' '.join(subsitite(parsed_rules, random.choice(v).split()))
            return response
        else:
            continue
    return '抱歉，请您再讲一遍'

In [31]:
get_response('I need HUAWEI', defined_patterns)

[('?X', 'HUAWEI')]


'Why do you need HUAWEI ?'

In [32]:
get_response('My mother told me something', defined_patterns)



[]
[('?X', 'mother')]


'How do you think about your mother ?'

# Segment Match ?*...


In [195]:
def is_segment(pattern):
    return pattern.startswith('?*') and all(i.isalpha() for i in pattern[2:])

fail = [True, None]

def pat_match_segment(pattern, saying):
    
    if not pattern or not saying: return []
   
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    
    elif is_segment(pattern[0]):
       
         if segment_match(pattern, saying) != fail:
            match, i = segment_match(pattern, saying)
            return [match] + pat_match_segment(pattern[1:], saying[i:])
    
    elif pattern[0] == saying[0]:
        return pat_match_segment(pattern[1:], saying[1:])
    else:
        return fail
    
def segment_match(pattern, sayings):
#      pat = pattern[0].replace('?*','?')
    pat, rest = pattern[0].replace('?*','?'), pattern[1:]
    if not rest:
        return (pat,sayings), len(sayings)
    
    for i,say in enumerate(sayings):
        #不仅第一个字母要相等，后面的都要一样才行，怎么判断后面的也要一样
        if rest[0] == say :
            return (pat, sayings[:i]), i
        
    return fail
        

In [192]:
segment_match('?*P is very good ?*Q'.split(), "My dog and my is a pig".split())

(('?P', ['My', 'dog', 'and', 'my']), 4)

In [196]:
cat_pattern = pat_match_segment('?*P is very good ?*Q'.split(), "My dog and my cat is very good and beautiful".split())
cat_pattern

[('?P', ['My', 'dog', 'and', 'my', 'cat']), ('?Q', ['and', 'beautiful'])]

In [197]:
#替换
def pat_seg_to_dict(parttern):
    return {k: ' '.join(v) if isinstance(v,list) else v for k,v in parttern } 

' '.join(subsitite(pat_seg_to_dict(cat_pattern), 'why do you think ?P is good ?Q ?'.split()))

'why do you think My dog and my cat is good and beautiful ?'

In [198]:
pat_match_segment('?*x hello ?*y'.split(), 'Dad I want an iphone'.split())

In [199]:
patterns_rlues = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really ?y', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

In [205]:
def get_seg_response(rules, saying):
    
    for k,v in rules.items():
        match_pattern = pat_match_segment(k.split(), saying.split())
        print(match_pattern)
        if not match_pattern or match_pattern == fail or True in match_pattern:
            continue
        else:
            return ' '.join(subsitite(pat_seg_to_dict(match_pattern), random.choice(v).split()))
    return "抱歉，请再讲一遍"
        
    

In [206]:
get_seg_response(patterns_rlues, 'Dad I want an iphone')

None
[('?x', ['Dad']), ('?y', ['an', 'iphone'])]


'Why do you want an iphone'

In [207]:
get_seg_response(patterns_rlues, 'Hi hello to you')

[('?x', ['Hi']), ('?y', ['to', 'you'])]


'Please state your problem'

In [244]:
get_seg_response(patterns_rlues, 'Hi I was Li')

None
[('?x', ['Hi']), True, None]
None
None
[('?x', ['Hi']), ('?y', ['Li'])]


'Were you really Li'

In [258]:
get_seg_response(patterns_rlues, 'baby I feel hot')

None
('?x', ['baby'])
[('?x', ['baby']), True, None]
None
None
('?x', ['baby'])
[('?x', ['baby']), True, None]
('?x', ['baby'])
('?y', ['hot'])
[('?x', ['baby']), ('?y', ['hot'])]


'Do you often feel hot ?'

## 问题2

### 改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [250]:
import jieba
import random

def cut(string):
    return ' '.join(jieba.cut(string))

fail = [True, None]

def is_segment(pattern):
    return pattern.startswith('?*') and all(i.isalpha() for i in pattern[2:])

def is_variable(pat):
    "return ?P 开头"
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:]) 


def pat_match_segment(pattern, saying):
    
    if not pattern or not saying: return []
   
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    
    elif is_segment(pattern[0]):
       
         if segment_match(pattern, saying) != fail:
            match, i = segment_match(pattern, saying)
            print(match)
            return [match] + pat_match_segment(pattern[1:], saying[i:])
    
    elif pattern[0] == saying[0]:
        return pat_match_segment(pattern[1:], saying[1:])
    else:
        return fail
    
def segment_match(pattern, sayings):
#      pat = pattern[0].replace('?*','?')
    pat, rest = pattern[0].replace('?*','?'), pattern[1:]
    if not rest:
        return (pat,sayings), len(sayings)
    
    for i,say in enumerate(sayings):
        #不仅第一个字母要相等，后面的都要一样才行，怎么判断后面的也要一样
        if rest[0] == say:
            return (pat, sayings[:i]), i
        
    return fail

def pat_chinseg_dict(parttern):
    return {k:''.join(v) if isinstance(v,list) else v for k,v in parttern}


In [273]:
def subsitite(pat_dic,saying):
    if not saying: return []
    return [pat_dic.get(saying[0],saying[0])] + subsitite(pat_dic,saying[1:])

def get_chinese_response(rules, saying):
    
    saying_token = cut(saying).split()
#     response_list = []
    for k,v in rules.items():
        key = cut(k).replace('? * x','?*x').replace('? * y','?*y').replace('? * z','?*z').split()
        
        seg_match = pat_match_segment(key, saying_token)
#         print(seg_match)
        if not seg_match or seg_match == fail or True in seg_match:
            continue
        else:
            seg_match_dict = pat_chinseg_dict(seg_match)
            print(seg_match_dict)
            response_token = cut(random.choice(v)).replace('? x','?x').replace('? y','?y').replace('? z','?*z').split()
            return (''.join(subsitite(seg_match_dict, response_token)))
            
    return '抱歉'
                
            

In [89]:
cut('这是一个测试')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.983 seconds.
Prefix dict has been built succesfully.


'这是 一个 测试'

In [252]:
chinese_rules={'?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']}

In [266]:
cut('?*x你为什么不?*y')

'? * x 你 为什么 不 ? * y'

In [267]:
cut('小红，你为什么不去游泳')

'小红 ， 你 为什么 不去 游泳'

In [274]:
get_chinese_response(chinese_rules, '小红，你为什么不游泳')
        

('?x', ['小红', '，'])
('?x', ['小红', '，'])
('?x', ['小红', '，'])
('?y', ['游泳'])
{'?x': '小红，', '?y': '游泳'}


'等我心情好了，我就游泳'

In [275]:
get_chinese_response(chinese_rules, '你似乎总是这样')

('?x', [])
('?x', [])
('?x', [])
('?x', ['你', '似乎'])
('?y', ['这样'])
{'?x': '你似乎', '?y': '这样'}


'例如什么时候?'

In [276]:
get_chinese_response(chinese_rules, '忧愁就像一江春水向东流')

('?x', ['忧愁'])
('?y', ['一江春水向东流'])
{'?x': '忧愁', '?y': '一江春水向东流'}


'忧愁和一江春水向东流真的有关系吗？'